In [1]:
from glob import glob
import os

In [2]:
texts = glob('call-centre-2-text/*')
len(texts)

3966

In [3]:
texts[:5]

['call-centre-2-text/app_1104_5208_phnd_cc-ins.TextGrid',
 'call-centre-2-text/app_1249_5498_phnd_cc-tel.TextGrid',
 'call-centre-2-text/app_0915_0007_phnd_cc-bnk.TextGrid',
 'call-centre-2-text/app_0956_4912_phnd_cc-ins.TextGrid',
 'call-centre-2-text/app_1301_5602_phnd_cc-ins.TextGrid']

In [4]:
os.path.split(texts[0])[1].replace('.TextGrid', '')

'app_1104_5208_phnd_cc-ins'

In [5]:
audios = glob('call-centre-2-audio/**/**.wav', recursive = True)
speakers = {}
for a in audios:
    speakers[os.path.split(a)[1].replace('.wav', '')] = a
    
len(speakers)

3966

In [7]:
import soundfile as sf
import textgrid

In [8]:
from tqdm import tqdm

directory = 'call-centre-2'

os.system(f'mkdir {directory}')
os.system(f'mkdir {directory}/wav')
os.system(f'mkdir {directory}/text')

256

In [9]:
def split(path):
    new_path = os.path.split(path)[1].replace('.TextGrid', '')
    y, sr = sf.read(speakers[new_path])
    index = 0
    path_textgrid = speakers[new_path]
    tg = textgrid.TextGrid.fromFile(path)
    for i in range(len(tg)):
        for k in range(len(tg[i])):
            try:
                splitted = y[int(sr * tg[i][k].minTime): int(sr * tg[i][k].maxTime)]
                sf.write(f'{directory}/wav/{new_path}-{i}-{k}.wav', splitted, sr)
                with open(f'{directory}/text/{new_path}-{i}-{k}.wav.txt', 'w') as fopen:
                    fopen.write(tg[i][k].mark)
            except Exception as e:
                print('nested exception', e)

In [10]:
split(texts[0])

In [11]:
wavs = glob(f'{directory}/wav/*.wav')
wavs[0]

'call-centre-2/wav/app_1136_5272_phnd_cc-ins-0-57.wav'

In [12]:
import IPython.display as ipd
ipd.Audio(wavs[0])

In [13]:
!cat {wavs[0].replace('/wav/', '/text/') + '.txt'}

(mm) may I know how is the application process like and how long will it take for the application to be approved

In [15]:
def loop(texts):
    texts = texts[0]
    for t in tqdm(texts):
        try:
            split(t)
        except Exception as e:
            print(e)
            pass

In [16]:
loop((texts[:1],))

100%|██████████| 1/1 [00:01<00:00,  1.08s/it]


In [ ]:
import mp

mp.multiprocessing(texts, loop, cores = 8, returned = False)

 17%|█▋        | 85/495 [02:04<11:48,  1.73s/it]

In [ ]:
# !tar --use-compress-program="pigz -0 -k " -cf part6-call-centre-2.tar.gz call-centre-2